In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint, TensorBoard
import datetime

In [2]:
(X_train, Y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [4]:
X_train = X_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
#print(type(X_train))

In [5]:
x_train=X_train[5000:]
x_valid=X_train[:5000]
y_train=Y_train[5000:]
y_valid=Y_train[:5000]

In [6]:
w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
x_test = x_test.reshape(x_test.shape[0], w, h, 1)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_valid = tf.keras.utils.to_categorical(y_valid, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [7]:
model = tf.keras.Sequential()

# Must define the input shape in the first layer of the neural network
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2,
          padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(
    filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Take a look at the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 32)          0

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)
model.fit(x_train,
          y_train,
          batch_size=64,
          epochs=10,
          validation_data=(x_valid, y_valid),
          callbacks=[checkpointer,tensorboard_callback])

Epoch 1/10
859/860 [============================>.] - ETA: 0s - loss: 0.6101 - accuracy: 0.7749
Epoch 1: val_loss improved from inf to 0.39379, saving model to model.weights.best.hdf5
860/860 [==============================] - 51s 58ms/step - loss: 0.6100 - accuracy: 0.7749 - val_loss: 0.3938 - val_accuracy: 0.8572
Epoch 2/10
859/860 [============================>.] - ETA: 0s - loss: 0.4282 - accuracy: 0.8456
Epoch 2: val_loss improved from 0.39379 to 0.32585, saving model to model.weights.best.hdf5
860/860 [==============================] - 48s 56ms/step - loss: 0.4282 - accuracy: 0.8456 - val_loss: 0.3259 - val_accuracy: 0.8888
Epoch 3/10
859/860 [============================>.] - ETA: 0s - loss: 0.3810 - accuracy: 0.8605
Epoch 3: val_loss improved from 0.32585 to 0.30976, saving model to model.weights.best.hdf5
860/860 [==============================] - 46s 53ms/step - loss: 0.3811 - accuracy: 0.8604 - val_loss: 0.3098 - val_accuracy: 0.8862
Epoch 4/10
859/860 [=====================

In [10]:
model.load_weights('model.weights.best.hdf5')

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.9071000218391418


In [12]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.


In [ ]:
# y_hat = model.predict(x_test)

# # Plot a random sample of 10 test images, their predicted labels and ground truth
# figure = plt.figure(figsize=(20, 8))
# for i, index in enumerate(np.random.choice(x_test.shape[0], size=15, replace=False)):
#     ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
#     # Display each image
#     ax.imshow(np.squeeze(x_test[index]))
#     predict_index = np.argmax(y_hat[index])
#     true_index = np.argmax(y_test[index])
#     # Set the title for each image
#     ax.set_title("{} ({})".format(fashion_mnist_labels[predict_index],
#                                   fashion_mnist_labels[true_index]),
#                  color=("green" if predict_index == true_index else "red"))